In [ ]:
!hf auth login --add-to-git-credential --token <hf_token>
!wandb login <wandb_token>

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!accelerate launch src/lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id={hf_user}/{dataset_name} \
  --batch_size=64\
  --steps=50000 \
  --save_freq=10000 \
  --output_dir=outputs/train/{job_name} \
  --job_name={job_name} \
  --policy.device=cuda \
  --policy.repo_id={hf_user}/{job_name} \
  --wandb.enable=true

In [ ]:
# resume training
!accelerate launch src/lerobot/scripts/train.py \
  --steps=60000 \
  --save_freq=10000 \
  --config_path=<path_to_checkpoint>/pretrained_model/train_config.json \
  --wandb.enable=true \
  --resume=true

In [ ]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model